<a href="https://colab.research.google.com/github/MahdiJahanbakht/cat-dog-classification-on-colab/blob/master/TransferLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 12.9 GB  | Proc size: 143.1 MB
GPU RAM Free: 11441MB | Used: 0MB | Util   0% | Total 11441MB


In [23]:

#' ' means CPU whereas '/device:G:0' means GPU
import tensorflow as tf
tf.test.gpu_device_name()


'/device:GPU:0'

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
!ls -a '/content/drive/My Drive/'

'Colab Notebooks'   Deep_Learning   kaggle.json


In [26]:
!pip install kaggle

In [27]:
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])

filename = "/content/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

Download 100%.


In [0]:
# !mkdir ~/.kaggle
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

In [29]:
!kaggle competitions list

ref                                                deadline             category            reward  teamCount  userHasEntered  
-------------------------------------------------  -------------------  ---------------  ---------  ---------  --------------  
digit-recognizer                                   2030-01-01 00:00:00  Getting Started  Knowledge       2624           False  
titanic                                            2030-01-01 00:00:00  Getting Started  Knowledge      10779           False  
house-prices-advanced-regression-techniques        2030-01-01 00:00:00  Getting Started  Knowledge       4411           False  
imagenet-object-localization-challenge             2029-12-31 07:00:00  Research         Knowledge         34           False  
competitive-data-science-predict-future-sales      2019-12-31 23:59:00  Playground           Kudos       2651           False  
two-sigma-financial-news                           2019-07-15 23:59:00  Featured          $100,000      

In [0]:
# kaggle datasets download -d biaiscience/dogs-vs-cats

import kaggle

kaggle.api.authenticate()

kaggle.api.dataset_download_files('biaiscience/dogs-vs-cats', path='/content/drive/My Drive/Deep_Learning/', unzip=False)

In [46]:
dataAddr = '/content/drive/My Drive/Deep_Learning/Datasets/'
import zipfile, os, shutil

# os.remove(zippedFile)

dataRoot = dataAddr + 'dogs-vs-cats'

# zipfile.ZipFile(dataRoot + '.zip').extractall(dataRoot)
# zipfile.ZipFile(dataRoot + '/test.zip').extractall(dataRoot + '/test')
# zipfile.ZipFile(dataRoot + '/train.zip').extractall(dataRoot + '/train')

# os.remove(dataRoot + '/train.zip')
# os.remove(dataRoot + '/test.zip')

source = dataAddr + 'zipfiles'
dest1 = source + '/cats'
dest2 = source + '/dogs'

files = os.listdir(source)

for f in files:
    if (f.startswith("cat") or f.startswith("Cat")):
        shutil.move(f, dest1)
    elif (f.startswith("dog") or f.startswith("Dog")):
        shutil.move(f, dest2)

FileNotFoundError: ignored

In [47]:
!ls '/content/drive/My Drive/Deep_Learning/Datasets/dogs-vs-cats/train/'

train


In [0]:
!pip install -q torch torchvision

In [0]:
import torch
from torch import nn, optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
import time

import matplotlib.pyplot as plt
import helper

from collections import OrderedDict
import time

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [0]:
transform = transforms.Compose([transforms.Resize(255),
                               transforms.CenterCrop(224),
                               transforms.ToTensor()])
dataAddr = 'C:\\Users\\Asus\\1_My_Notebooks\\PyTorch\\Data\\Cat_Dog_data\\'
batch_size = 64


train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

trainset = datasets.ImageFolder(dataAddr+'train', transform=train_transforms)
testset = datasets.ImageFolder(dataAddr+'test',transform=test_transforms)

trainloader = torch.utils.data.DataLoader(trainset,batch_size=batch_size,shuffle=True)
testloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size)

In [0]:
data_iter = iter(testloader)

images, labels = next(data_iter)
fig, axes = plt.subplots(figsize=(10,4), ncols=4)
for ii in range(4):
    ax = axes[ii]
    helper.imshow(images[ii], ax=ax, normalize=True)

In [0]:
model = models.densenet121(pretrained=True)
model

In [0]:
for param in model.parameters():
    param.requires_grad = False

classifier = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(1024, 500)),
    ('relu', nn.ReLU()),
    ('fc2', nn.Linear(500, 2)),
    ('output', nn.LogSoftmax(dim=1))
    ]))
model.classifier = classifier

for device in ['cpu', 'cuda']:
    criterion = nn.NLLLoss()
    optimizer = optim.Adam(model.classifier.parameters(), lr= 0.001)

    model.to(device)

    for ii, (inputs, labels) in enumerate(trainloader):
        inputs, labels = inputs.to(device), labels.to(device)
        start = time.time()
        outputs = model.forward(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if ii == 3:
            break
    print(f"Device = {device}; Time per batch: {(time.time() - start)/3:.3f} seconds")

In [0]:
# Use Gpu if it's available
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

model = models.resnet50(pretrained=True)

# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False

classifier = nn.Sequential(nn.Linear(2048,512),
                          nn.ReLU(),
                          nn.Dropout(p=0.2),
                          nn.Linear(512,2),
                          nn.LogSoftmax(dim=1))
model.fc = classifier

criterion = nn.NLLLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.003)

model.to(device)

In [0]:
epochs = 1
steps = 0
running_loss = 0
print_every = 5
train_losses, test_losses = [], []

for epoch in range(epochs):
    for inputs, labels in trainloader:
        steps += 1
        # Move input and label tensors to the default device
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        logps = model.forward(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            model.eval()
            with torch.no_grad():
                for inputs, labels in testloader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    logps = model.forward(inputs)
                    batch_loss = criterion(logps, labels)
                    
                    test_loss += batch_loss.item()
                    
                    # Calculate accuracy
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
            
            train_losses.append(running_loss/print_every)
            test_losses.append(test_loss/len(testloader))
            
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(testloader):.3f}.. "
                  f"Test accuracy: {accuracy/len(testloader):.3f}")
            running_loss = 0
            model.train()

In [0]:
plt.plot(train_losses, label = 'Train loss')
plt.plot(test_losses, label = 'Test loss')
plt.legend(frameon = False)